# Initializing Database

In [1]:
import redis
r = redis.Redis(host = "localhost",port=6379,db=0,)
r.ping()

True

In [5]:
for key in r.scan_iter():
    print(key)

# Splitting

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from pipeline import DocumentExtractor

extractor = DocumentExtractor()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=200
)
loader = extractor.extract_content("docs/soil.pdf")
chunks = text_splitter.create_documents([loader["content"]])
print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", 'samples/soil.pdf')

Using device: cuda
Done preprocessing. Created 8 chunks of the original pdf samples/soil.pdf


In [8]:
loader

{'content': 'Soil Mechanics Lectures\nch-1\nAsst. Prof. Rakshak Dhungel, nec\n\nEvaluation\nTheory\nPractical\nTotal\nSessional\n30\n20\n50\nFinal\n50\n-\n50\nTotal\n80\n20\n100\n\nInternal Marks Distribution\n• Attendance\n• Tutorials\n• Lab report preparation\n• MCQ and Viva                        \n \n    \n• UT and Assessment Exam           \n\nText books\n\nTextbooks\n\nSyllabus\n• Chapter 1 – Introduction\n• Chapter 2 – Phase relationship and Index        \n   \n     properties of soils\n• Chapter 3 – Soil Identification and classification\n• Chapter 4 – Introduction to clay minerals\n• Chapter 5 – Compaction of soil\n• Chapter 6 – Effective stress, capillarity and \n \n \n     permeability\n• Chapter 7 – Seepage analysis through soil\n\nSyllabus\n• Chapter 8 – Vertical stress below applied load\n• Chapter 9 – Compressibility and consolidation   \n      of soil\n• Chapter 10 – Shear strength of soil\n• Chapter 11 – Stability of slopes\n\nPracticals\n• Particle size distribution t

# Embedding Using all-MiniLM-L6-v2

In [9]:
from redisvl.utils.vectorize import HFTextVectorizer
import pandas as pd
from tqdm.auto import tqdm
import os

hf = HFTextVectorizer("sentence-transformers/all-MiniLM-L6-v2")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Embed each chunk content
embeddings = hf.embed_many([chunk.page_content for chunk in chunks])

# Check to make sure we've created enough embeddings, 1 per document chunk
len(embeddings) == len(chunks)

True

In [10]:
from redis import Redis
from redisvl.index import SearchIndex

index_name = "redisvl"

schema = {
  "index": {
    "name": index_name,
    "prefix": "chunk"
  },
  "fields": [
    {
        "name": "chunk_id",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "content",
        "type": "text"
    },
    {
        "name": "text_embedding",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
}

In [11]:
# create an index from schema and the client
index = SearchIndex.from_dict(schema)
index.set_client(r)
index.create(overwrite=True, drop=True)

19:19:25 redisvl.index.index INFO   Index already exists, overwriting.


In [12]:
# load expects an iterable of dictionaries
from redisvl.redis.utils import array_to_buffer

data = [
    {
        'chunk_id': i,
        'content': chunk.page_content,
        # For HASH -- must convert embeddings to bytes
        'text_embedding': array_to_buffer(embeddings[i], dtype='float32')
    } for i, chunk in enumerate(chunks)
]

# RedisVL handles batching automatically
keys = index.load(data, id_field="chunk_id")

In [13]:
from redisvl.query import VectorQuery

query = "what is soil mechanics?"

query_embedding = hf.embed(query)

vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=3,
    return_fields=["chunk_id", "content"],
    return_score=True
)

# show the raw redis query
str(vector_query)

'*=>[KNN 3 @text_embedding $vector AS vector_distance] RETURN 3 chunk_id content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 3'

In [14]:
# execute the query with RedisVL
result=index.query(vector_query)

# view the results
pd.DataFrame(result)

,id,vector_distance,chunk_id,content
0,chunk:1,0.284770131111,1,"• To the civil engineers, soil is any un-cemen..."
1,chunk:2,0.390098035336,2,• Soil mechanics equips engineers with scienti...
2,chunk:3,0.467613518238,3,2. Earthen Dams\n• Dam construction provide wa...


In [15]:
result

[{'id': 'chunk:1',
  'vector_distance': '0.284770131111',
  'chunk_id': '1',
  'content': '• To the civil engineers, soil is any un-cemented or \nweakly cemented accumulation of solid grain \nparticles, the void space between the particles \ncontaining water and/or air.\n\uf0d8grains(mineral grains, rock fragments etc)\n\uf0d8Void space(water and air)\n\uf0d8The void spaces are changed by changes in field \ncondition.\nFig: Soil grain structure\n\uf0d8Engineers assess soil properties, such as its \nbearing \ncapacity, \ncompaction \ncharacteristics, \npermeability, \nand \nsettlement \npotential, \nto \ndesign \nand \nconstruct structures like buildings, roads, \nbridges, and dams that are safe and stable \non the soil beneath them.\n\n• Soil consists of a multiphase aggregation of solid \nparticles, water, and air. \n• This fundamental composition gives rise to unique \nengineering properties.\n• The \ndescription \nof \nits \nmechanical \nbehaviour(strength, permeability, seepage etc

In [16]:
def retrieve_documents(query: str) -> list:
    # Create embedding for the query
    query_embedding = hf.embed(query)
    
    # Create vector query
    vector_query = VectorQuery(
        vector=query_embedding,
        vector_field_name="text_embedding",
        num_results=3,
        return_fields=["content"],
        return_score=True
    )
    
    # Execute query
    results = index.query(vector_query)
    return results
    
results = retrieve_documents(query)

context = "\n".join([result["content"] for result in results])
df = pd.DataFrame(results)
df.head()

,id,vector_distance,content
0,chunk:1,0.284770131111,"• To the civil engineers, soil is any un-cemen..."
1,chunk:2,0.390098035336,• Soil mechanics equips engineers with scienti...
2,chunk:3,0.467613518238,2. Earthen Dams\n• Dam construction provide wa...


In [17]:
import os
from chatbot import ChatBot
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file
model = ChatBot(api_token=os.getenv('HUGGINGFACE_API_KEY'))

In [5]:
model.query("Hello")

'Hello! How can I assist you today? Feel free to ask me anything or let me know if you need help with a specific topic.'

In [24]:
chat_history = [
    {"role": "user", "content": "Hello"},
    {"role": "bot", "content": "Hi there! How can I help you today?"}
]
query = "What is soil mechanics?"
context = "\n".join([result["content"] for result in results])

def retrieve_documents(query: str) -> list:
    query_embedding = hf.embed(query)
    
    vector_query = VectorQuery(
        vector=query_embedding,
        vector_field_name="text_embedding",
        num_results=3,
        return_fields=["content"],
        return_score=True
    )
    
    # Execute query
    results = index.query(vector_query)
    return results
    
results = retrieve_documents(query)

def generate_response(chat_history, query):
    results = retrieve_documents(query)
    context = "\n".join([result["content"] for result in results])
    chat_history.append({"role": "user", "content": query})
    response = model.generate_response(context, chat_history)
    return {"role": "bot", "content": response}

response = generate_response(chat_history, query)
response

[{'role': 'system', 'content': 'You are a helpful AI assistant that provides clear and concise information based on the given context and chat history.'}, {'role': 'user', 'content': 'Context: '}, {'role': 'user', 'content': 'Hello'}, {'role': 'bot', 'content': 'Hi there! How can I help you today?'}, {'role': 'user', 'content': 'What is soil mechanics?'}]


{'role': 'bot',
 'content': "Soil mechanics is a branch of engineering that deals with the behavior of soil under various conditions, particularly in relation to construction and civil engineering projects. It involves the study of the physical properties of soil, such as its strength, compressibility, and permeability, and how these properties affect the soil's behavior under loads, water, and other environmental factors. Soil mechanics is crucial for designing foundations, earth structures, and for understanding slope stability, among other applications."}

In [19]:
# Join the content from the search results
"\n\n".join([doc['content'] for doc in result[0:3]])

'• To the civil engineers, soil is any un-cemented or \nweakly cemented accumulation of solid grain \nparticles, the void space between the particles \ncontaining water and/or air.\n\uf0d8grains(mineral grains, rock fragments etc)\n\uf0d8Void space(water and air)\n\uf0d8The void spaces are changed by changes in field \ncondition.\nFig: Soil grain structure\n\uf0d8Engineers assess soil properties, such as its \nbearing \ncapacity, \ncompaction \ncharacteristics, \npermeability, \nand \nsettlement \npotential, \nto \ndesign \nand \nconstruct structures like buildings, roads, \nbridges, and dams that are safe and stable \non the soil beneath them.\n\n• Soil consists of a multiphase aggregation of solid \nparticles, water, and air. \n• This fundamental composition gives rise to unique \nengineering properties.\n• The \ndescription \nof \nits \nmechanical \nbehaviour(strength, permeability, seepage etc.) \nrequires some of the most classic principles of \nengineering mechanics.\n• Soil Mech

In [20]:
from redis.asyncio import Redis as AsyncRedis
from redisvl.index import AsyncSearchIndex
REDIS_URL = "redis://localhost:6379/0"
client = AsyncRedis.from_url(REDIS_URL)
async_index = AsyncSearchIndex.from_dict(schema)
await async_index.set_client(client)

In [21]:
from chatbot import ChatBot

In [22]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv()  # Load environment variables
model = ChatBot(api_token=os.getenv('HUGGINGFACE_API_KEY'))

async def answer_question(index: AsyncSearchIndex, query: str):
    """Answer the user's question"""

    SYSTEM_PROMPT = """You are a helpful financial analyst assistant that has access
    to public financial 10k documents in order to answer users questions about company
    performance, ethics, characteristics, and core information.
    """

    query_vector = hf.embed(query)
    # Fetch context from Redis using vector search
    context = await retrieve_context(index, query_vector)
    # Generate contextualized prompt and feed to OpenAI
    response = await model.query(promptify(query, context))
    # Response provided by LLM
    return response.choices[0].message.content


async def retrieve_context(async_index: AsyncSearchIndex, query_vector) -> str:
    """Fetch the relevant context from Redis using vector search"""
    results = await async_index.query(
        VectorQuery(
            vector=query_vector,
            vector_field_name="text_embedding",
            return_fields=["content"],
            num_results=3
        )
    )
    content = "\n".join([result["content"] for result in results])
    return content


def promptify(query: str, context: str) -> str:
    return f'''Use the provided context below derived from public financial
    documents to answer the user's question. If you can't answer the user's
    question, based on the context; do not guess. If there is no context at all,
    respond with "I don't know".

    User question:

    {query}

    Helpful context:

    {context}

    Answer:
    '''

In [24]:
import huggingface_hub
from typing import List, Dict
from huggingface_hub import InferenceClient
class ChatBot:
    def __init__(self, api_token: str):
        self.client = InferenceClient(api_key=api_token)
        self.model = "Qwen/Qwen2.5-72B-Instruct"

    def generate_response(self, context: str, chat_history: List[Dict[str, str]]) -> str:
        """
        Generate a response based on the provided context and chat history.
        This method constructs a message sequence including a system message, 
        the given context, and the chat history, then uses an AI model to 
        generate a response.
        Args:
            context (str): The current context or query from the user.
            chat_history (List[Dict[str, str]]): A list of dictionaries containing 
                                                 the chat history with keys "role" 
                                                 and "content".
        Returns:
            str: The generated response from the AI model.
        """
        
        
        system_msg = "You are a helpful AI assistant that provides clear and concise information based on the given context and chat history."
        
        formatted_history = []
        for msg in chat_history:
            formatted_history.append({"role": msg["role"], "content": msg["content"]})
    
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": f"Context: {context}"},
        ]
        messages.extend(formatted_history)
        print(messages)
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7,
            max_tokens=1024,
            top_p=0.9
        )
        
        return response.choices[0].message.content
    
    def query(self, query:str):
        messages = [
            { "role": "user", "content": query }
        ]

        completion = client.chat.completions.create(
            model=self.model, 
            messages=messages, 
            temperature=0.5,
            max_tokens=2048,
            top_p=0.7
        )

        return completion.choices[0].message.content

In [25]:
import asyncio

result = await answer_question(async_index, "what is soil mechanics?")
print(result)

NameError: name 'client' is not defined

In [25]:
result.

<coroutine object answer_question at 0x000001FD38209540>